In [1]:
import pytesseract
from PIL import Image, ImageEnhance, ImageOps
from pdf2image import convert_from_path
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import tempfile

# Input PDF file path
# pdf_path = '/home/om_prakash/Desktop/5/test3.pdf'
# pdf_path = '/home/om_prakash/Desktop/5/op1.pdf'
# pdf_path = '/home/om_prakash/Desktop/5/output.pdf'
# pdf_path = '/home/om_prakash/Desktop/5/png2pdf.pdf'
# pdf_path = '/home/om_prakash/Desktop/5/Check_IT/Degree_Seedha.jpg'



# Output PDF file path
output_pdf_path = '/home/om_prakash/Desktop/5/op.pdf'

# Convert PDF pages to images
images = convert_from_path(pdf_path)

# Create a PDF writer object
pdf_writer = canvas.Canvas(output_pdf_path, pagesize=letter)

# Process each image
for page_number, image in enumerate(images):
    # Preprocess the image for better OCR results
    enhancer = ImageEnhance.Contrast(image)
    enhanced_image = enhancer.enhance(1.5)  # Increase contrast
    inverted_image = ImageOps.invert(enhanced_image)  # Invert colors

    # Perform OSD (Orientation and Script Detection) on the original image
    osd = pytesseract.image_to_osd(image)

    # Extract rotation angle and script orientation from OSD output
    rotation_angle = int(osd.split('Rotate: ')[1].split('\n')[0])
    print("Observed Rotation Angle for Page {}: {}".format(page_number + 1, rotation_angle))

    # Determine the adjustment angle based on script orientation
    if rotation_angle == 90:
        adjustment_angle = -90  # Rotate anti-clockwise by 90 degrees
    elif rotation_angle == 270:
        adjustment_angle = -90  # Rotate clockwise by 90 degrees
    else:
        adjustment_angle = rotation_angle

    # Rotate the image by the adjustment angle
    adjusted_image = image.rotate(adjustment_angle, expand=True)

    # Save the adjusted image to a temporary file
    with tempfile.NamedTemporaryFile(suffix='.png') as temp_file:
        adjusted_image.save(temp_file.name)

        # Add the adjusted image as a new page to the PDF writer
        pdf_writer.setPageSize((adjusted_image.width, adjusted_image.height))
        pdf_writer.drawImage(temp_file.name, 0, 0)

        # Show page number on the current page
        pdf_writer.setFont("Helvetica", 10)
        pdf_writer.drawString(10, 10, "Page {}".format(page_number + 1))

        # Save the page
        pdf_writer.showPage()

# Save the output PDF file
pdf_writer.save()

print("Output PDF generated successfully.")


Observed Rotation Angle for Page 1: 270
Output PDF generated successfully.
